In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from datetime import datetime
import time
from selenium.webdriver.common.keys import Keys
import re

In [9]:
def login_netkeiba():
    service = Service('C:\\Users\\ryo\\AppData\\Local\\SeleniumBasic\\chromedriver.exe')
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=service, options=options)

    # ログイン
    url_login = "https://regist.netkeiba.com/account/?pid=login"
    driver.get(url_login)
    driver.find_element(By.NAME, "login_id").send_keys("keibaaitool@gmail.com")
    pw_field = driver.find_element(By.NAME, "pswd")
    pw_field.send_keys("keibaai")
    pw_field.send_keys(Keys.ENTER)
    time.sleep(3)

    # 日付
    today = datetime.now()
    yyyy = today.strftime("%Y")
    mmdd = today.strftime("%m%d")

    # 対象競馬場とレース
    place_codes = [
        "35", "45", "55", "65", "46", "54", "44",
        "47", "50", "30", "36", "43", "48", "42"
    ]
    race_numbers = [f"{i:02d}" for i in range(1, 13)]

    for place_code in place_codes:
        for race_num in race_numbers:
            race_id = f"{yyyy}{place_code}{mmdd}{race_num}"
            url = f"https://nar.netkeiba.com/race/shutuba.html?race_id={race_id}"
            driver.get(url)
            time.sleep(3)

            try:
                # 競馬場
                racecourse_elem = driver.find_element(By.CSS_SELECTOR,
                    "#Netkeiba_Race_Nar_Shutuba > div.Wrap.fc > div.RaceColumn01 > div > div.RaceMainColumn > div.RaceList_NameBox > div.RaceList_Item02 > div.RaceData02 > span:nth-child(2)")
                racecourse = racecourse_elem.text
                # R
                race_num_elem = driver.find_element(By.CSS_SELECTOR,
                    "#Netkeiba_Race_Nar_Shutuba > div.Wrap.fc > div.RaceColumn01 > div > div.RaceMainColumn > div.RaceList_NameBox > div.RaceList_Item01 > div")
                race_num = race_num_elem.text
                # 頭数
                num_horses_elem = driver.find_element(By.CSS_SELECTOR,
                    "#Netkeiba_Race_Nar_Shutuba > div.Wrap.fc > div.RaceColumn01 > div > div.RaceMainColumn > div.RaceList_NameBox > div.RaceList_Item02 > div.RaceData02 > span:nth-child(5)")
                num_horses_text = num_horses_elem.text
                num_horses = int(num_horses_text.replace("頭", "").strip())
                # レース情報（まとめられている）
                race_info_xpath = '//*[@id="Netkeiba_Race_Nar_Shutuba"]/div[1]/div[2]/div/div[1]/div[3]/div[2]/div[2]'
                race_info_elem = driver.find_element(By.XPATH, race_info_xpath)
                race_info_text = race_info_elem.text.strip()
                # "/"で分割
                parts = [p.strip() for p in race_info_text.split("/")]            
                # 発送時刻を抽出
                race_time = parts[0].replace("発走", "").strip()
                ground_info = parts[1]
                # グラウンドを抽出
                ground_match = re.search(r"(芝|ダ)", ground_info)
                ground = ground_match.group(1) if ground_match else "不明"
                # 距離を抽出
                distance_match = re.search(r"(\d+)m", ground_info)
                distance = int(distance_match.group(1)) if distance_match else -1
                # 向きを抽出
                direction_match = re.search(r"\((左|右)\)", ground_info)
                direction = direction_match.group(1) if direction_match else "不明"
                # 出力確認
                print(f"\n📄 Race ID: {race_id} | 競馬場: {racecourse} | R: {race_num} | 頭数: {num_horses} | 🕓 発走時刻: {race_time} | 🏇 グラウンド: {ground} | 📏 距離: {distance}m | ↩ 回り: {direction}")
                # 馬番・馬名取得
                # 馬番・馬名・馬名リンク取得
                for i in range(1, num_horses + 1):
                    try:
                        # 馬番
                        umaban_xpath = f'//*[@id="tr_{i}"]/td[2]'
                        umaban = driver.find_element(By.XPATH, umaban_xpath).text.strip()
                        # 馬名とそのリンク
                        horse_link_xpath = f'/html/body/div[1]/div[3]/div[3]/table/tbody/tr[{i}]/td[4]/div/div/span[2]/a'
                        horse_link_elem = driver.find_element(By.XPATH, horse_link_xpath)
                        horse_name = horse_link_elem.text.strip()
                        horse_href = horse_link_elem.get_attribute("href")
                        # 騎手名とそのリンク
                        jockey_link_xpath = f'//*[@id="tr_{i}"]/td[7]/span/a'
                        jockey_link_elem = driver.find_element(By.XPATH, jockey_link_xpath)
                        jockey_name = jockey_link_elem.text.strip()
                        jockey_href = jockey_link_elem.get_attribute("href")
                        print(f"  馬番{i}: {umaban} | 馬名: {horse_name} | リンク: {horse_href} | 騎手名: {jockey_name} | リンク: {jockey_href}")
                
                    except Exception as e:
                        print(f"  馬番{i}: データ取得失敗 ({e})")

            except Exception as e:
                print(f"❌ Race ID: {race_id} データ取得に失敗: {e}")
                break
    driver.quit()

login_netkeiba()


❌ Race ID: 202535101501 データ取得に失敗: invalid literal for int() with base 10: ''

📄 Race ID: 202545101501 | 競馬場: 川崎 | R: 1R | 頭数: 12 | 🕓 発走時刻: 14:45 | 🏇 グラウンド: ダ | 📏 距離: 900m | ↩ 回り: 左
  馬番1: 1 | 馬名: スマートドラフト | リンク: https://db.netkeiba.com/horse/2019101309 | 騎手名: 赤津和希 | リンク: https://db.netkeiba.com/jockey/result/recent/05561
  馬番2: 2 | 馬名: パトウォン | リンク: https://db.netkeiba.com/horse/2019103836 | 騎手名: 櫻井光輔 | リンク: https://db.netkeiba.com/jockey/result/recent/05563
  馬番3: 3 | 馬名: ササキンポピー | リンク: https://db.netkeiba.com/horse/2021100463 | 騎手名: 古岡勇樹 | リンク: https://db.netkeiba.com/jockey/result/recent/05606
  馬番4: 4 | 馬名: ダイナマイトブル | リンク: https://db.netkeiba.com/horse/2020101068 | 騎手名: 達城龍次 | リンク: https://db.netkeiba.com/jockey/result/recent/05540
  馬番5: 5 | 馬名: シンキングセンティ | リンク: https://db.netkeiba.com/horse/2019100703 | 騎手名: 岡田大 | リンク: https://db.netkeiba.com/jockey/result/recent/05348
  馬番6: 6 | 馬名: ビギニングドリーム | リンク: https://db.netkeiba.com/horse/2018102352 | 騎手名: 神尾香澄 | リンク: https://db.netkeiba.c

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=141.0.7390.77)
Stacktrace:
	GetHandleVerifier [0x0x7ff7f802e955+80021]
	GetHandleVerifier [0x0x7ff7f802e9b0+80112]
	(No symbol) [0x0x7ff7f7db060f]
	(No symbol) [0x0x7ff7f7d882f1]
	(No symbol) [0x0x7ff7f7e388be]
	(No symbol) [0x0x7ff7f7e58fa2]
	(No symbol) [0x0x7ff7f7e31003]
	(No symbol) [0x0x7ff7f7df95d1]
	(No symbol) [0x0x7ff7f7dfa3f3]
	GetHandleVerifier [0x0x7ff7f82edd0d+2960461]
	GetHandleVerifier [0x0x7ff7f82e7fca+2936586]
	GetHandleVerifier [0x0x7ff7f8308a07+3070279]
	GetHandleVerifier [0x0x7ff7f804843e+185214]
	GetHandleVerifier [0x0x7ff7f804fe8f+216527]
	GetHandleVerifier [0x0x7ff7f8037b94+117460]
	GetHandleVerifier [0x0x7ff7f8037d4f+117903]
	GetHandleVerifier [0x0x7ff7f801dc28+11112]
	BaseThreadInitThunk [0x0x7ffd9e6ae8d7+23]
	RtlUserThreadStart [0x0x7ffd9f0ec53c+44]
